In [51]:
import pandas as pd
import os, requests, zipfile, warnings, time
from io import BytesIO
import datetime
warnings.filterwarnings('ignore')

In [30]:
test_url = 'http://data.gdeltproject.org/gdeltv2/20250219000000.gkg.csv.zip'
base_url = 'http://data.gdeltproject.org/gdeltv2/'

In [ ]:
def parse_data(data):
    df = pd.read_csv(
        data,
        delimiter='\t',
        encoding='utf-8',
        low_memory=False,
        header=None,
        usecols=[0, 1, 2, 3, 4, 7, 9, 13, 15, 17, 25, 26],
        names=[
            'GKGRECORDID',
            'V2.1DATE',
            'V2SOURCECOLLECTIONIDENTIFIER',
            'V2SOURCECOMMONNAME',
            'V2DOCUMENTIDENTIFIER',
            'V1THEMES',
            'V1LOCATIONS',
            'V1ORGANIZATIONS',
            'V1.5TONE',
            'V2GCAM',
            'V2.1TRANSLATIONINFO',
            'V2EXTRASXML'
        ]
    )
    return(df)

In [35]:
def download_unzip(url):
    response = requests.get(url)

    # Create a BytesIO object from the downloaded content
    zip_bytes = BytesIO(response.content)

    # Open the zip file in memory
    with zipfile.ZipFile(zip_bytes, 'r') as zip_file:
        # Get the first file in the zip
        first_file = zip_file.namelist()[0]
        
        # Read the file content directly into memory
        with zip_file.open(first_file) as file:
            return(parse_data(file))

In [36]:
download_unzip(test_url)

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
0,20250219000000-0,20250219000000,1,mmm-online.com,https://www.mmm-online.com/features/ai-encroac...,TAX_FNCACT;TAX_FNCACT_CMO;TAX_ECON_PRICE;WB_62...,1#France#FR#FR#46#2#FR;1#Canada#CA#CA#60#-96#CA,pfizer;bristol myers squibb;martin agency;amge...,"1.02657004830918,2.89855072463768,1.8719806763...","wc:1503,c1.2:3,c12.1:122,c12.10:143,c12.12:46,...",NaN,<PAGE_LINKS>https://www.interpublic.com/news/s...
1,20250219000000-1,20250219000000,1,nbcrightnow.com,https://www.nbcrightnow.com/news/crystal-apple...,EDUCATION;TAX_FNCACT;TAX_FNCACT_EDUCATORS;WB_4...,NaN,finley elementary school;desert oasis high sch...,"9.41176470588235,9.41176470588235,0,9.41176470...","wc:151,c12.1:11,c12.10:6,c12.13:2,c12.14:4,c12...",NaN,<PAGE_AUTHORS>Thomas Metcalf;NonStop Local Dig...
2,20250219000000-2,20250219000000,1,centralwesterndaily.com.au,https://www.centralwesterndaily.com.au/story/8...,KILL;CRISISLEX_T03_DEAD;DISASTER_FIRE;CRISISLE...,"4#Sydney, New South Wales, Australia#AS#AS02#-...",australian associated,"-2.87610619469027,1.10619469026549,3.982300884...","wc:410,c12.1:23,c12.10:31,c12.12:16,c12.13:7,c...",NaN,<PAGE_PRECISEPUBTIMESTAMP>20250218225100</PAGE...
3,20250219000000-3,20250219000000,1,wlfi.com,https://www.wlfi.com/news/ive-got-many-levers-...,LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;ELECTION...,"3#Terre Haute, Indiana, United States#US#USIN#...",braun,"-0.681431005110733,2.72572402044293,3.40715502...","wc:557,c12.1:34,c12.10:62,c12.12:20,c12.13:22,...",NaN,<PAGE_AUTHORS>Ben Verbanic;By: Ben Verbanic</P...
4,20250219000000-4,20250219000000,1,ksat.com,https://www.ksat.com/news/politics/2025/02/18/...,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_KANSAS;B...,"2#Kansas, United States#US#USKS#38.5111#-96.80...",NaN,"-3.96039603960396,2.64026402640264,6.600660066...","wc:273,c12.1:28,c12.10:31,c12.12:9,c12.13:5,c1...",NaN,<PAGE_LINKS>https://apnews.com/article/transge...
...,...,...,...,...,...,...,...,...,...,...,...,...
1827,20250219000000-1827,20250219000000,1,dailyherald.com,https://www.dailyherald.com/20250218/crime/man...,TAX_FNCACT;TAX_FNCACT_MAN;TRIAL;KILL;CRIME_ILL...,NaN,NaN,"-9.40594059405941,1.48514851485149,10.89108910...","wc:174,c12.1:16,c12.10:14,c12.12:7,c12.13:4,c1...",NaN,<PAGE_AUTHORS>Susan Sarkauskas</PAGE_AUTHORS><...
1828,20250219000000-1828,20250219000000,1,theboxhouston.com,https://theboxhouston.com/11602732/dangelo-mee...,TAX_FNCACT;TAX_FNCACT_SINGER;CRISISLEX_C07_SAF...,"3#Philadelphia, Pennsylvania, United States#US...",mann center,"4.4,5.6,1.2,6.8,21.2,0.4,216","wc:216,c1.2:1,c12.1:17,c12.10:18,c12.12:3,c12....",NaN,<PAGE_LINKS>https://therootspicnic.com/;https:...
1829,20250219000000-1829,20250219000000,1,pr-inside.com,https://www.pr-inside.com/contact-levi-korsins...,ALLIANCE;LEGISLATION;WB_845_LEGAL_AND_REGULATO...,1#United States#US#US#39.828175#-98.5795#US,walgreens boots alliance inc;news network;walg...,"-4.32801822323462,2.27790432801822,6.605922551...","wc:348,c1.3:1,c12.1:22,c12.10:43,c12.12:11,c12...",NaN,<PAGE_LINKS>http://Esq.33;http://Esq.Ed;http:/...
1830,20250219000000-1830,20250219000000,1,ncpa.org,https://www.ncpa.org/newsroom/qam/2025/02/19/s...,LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDENT;USPEC_P...,1#Japan#JA#JA#36#138#JA;1#United States#US#US#...,national archives;supreme court,"-5.18518518518519,1.48148148148148,6.666666666...","wc:247,c12.1:15,c12.10:27,c12.12:12,c12.13:11,...",NaN,<PAGE_LINKS>https://www.archives.gov/education...


In [45]:
def generate_url_list(start_dt, end_dt, increment_minutes=15):
    url_list = []
    current_dt = start_dt
    
    while current_dt <= end_dt:
        # Format as YYYYMMDDHHMMSS
        datetime_str = current_dt.strftime('%Y%m%d%H%M%S')
        url_list.append(f'{base_url}{datetime_str}.gkg.csv.zip')
        
        # Add increment
        current_dt += datetime.timedelta(minutes=increment_minutes)
    
    return url_list

In [47]:
start = datetime.datetime(2018, 1, 1, 0, 0, 0)
end = datetime.datetime(2025, 5, 1, 0, 0, 0)
urls = generate_url_list(start, end, 15)
print(urls[0:10])

['http://data.gdeltproject.org/gdeltv2/20180101000000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101001500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101003000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101004500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101010000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101011500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101013000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101014500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101020000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101021500.gkg.csv.zip']


In [48]:
# function to chunk datetime_strings into groups of 100
def chunk_datetime_strings(urls, chunk_size=100):
    return [urls[i:i + chunk_size] for i in range(0, len(urls), chunk_size)]
chunked_datetime_strings = chunk_datetime_strings(urls, 100)

In [ ]:
def download_chunk(chunk):
    # List to store successfully downloaded DataFrames
    df_list = []
    # Create log file name with current datetime
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"failed_downloads_{current_time}.txt"
    
    # Timing variables
    iteration_times = []
    
    for i, url in enumerate(chunk):
        start_time = time.time()
        
        try:
            # Try to download and unzip the file, then parse it into a DataFrame
            df = download_unzip(url)
            df_list.append(df)
        except Exception as e:
            # If download or parsing fails, log the URL and error message
            with open(log_file, "a") as f:
                f.write(f"{url}\n")
            print(f"Failed to download or parse {url}.")

        time.sleep(1)  # Sleep to avoid overwhelming the server
        
        # Calculate iteration time
        iteration_time = time.time() - start_time
        iteration_times.append(iteration_time)
        
        # Print average every 10 iterations
        if (i + 1) % 10 == 0:
            avg_time = sum(iteration_times) / len(iteration_times)
            print(f"Processed {i + 1}/{len(chunk)} URLs. Average time per iteration: {avg_time:.2f} seconds")
            
    if df_list:
        # Concatenate all DataFrames if any were successfully downloaded
        return pd.concat(df_list, ignore_index=True)
    else:
        # If none succeeded, print a message and return None
        print(f"None of the URLs worked.")
        return None

In [54]:
test=urls[0:100]
df=download_chunk(test)
df

Processed 10/100 URLs. Average time per iteration: 1.84 seconds
Processed 20/100 URLs. Average time per iteration: 1.80 seconds
Processed 30/100 URLs. Average time per iteration: 1.76 seconds
Failed to download or parse http://data.gdeltproject.org/gdeltv2/20180101074500.gkg.csv.zip: File is not a zip file
Processed 40/100 URLs. Average time per iteration: 1.72 seconds
Processed 50/100 URLs. Average time per iteration: 1.71 seconds
Processed 60/100 URLs. Average time per iteration: 1.70 seconds
Processed 70/100 URLs. Average time per iteration: 1.72 seconds
Failed to download or parse http://data.gdeltproject.org/gdeltv2/20180101191500.gkg.csv.zip: File is not a zip file
Processed 80/100 URLs. Average time per iteration: 1.71 seconds
Processed 90/100 URLs. Average time per iteration: 1.71 seconds
Processed 100/100 URLs. Average time per iteration: 1.73 seconds


,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
0,20180101000000-0,20180101000000,1,stamfordadvocate.com,http://www.stamfordadvocate.com/news/politics/...,TAX_FNCACT;TAX_FNCACT_CHIEF;WB_696_PUBLIC_SECT...,"2#Florida, United States#US#USFL#27.8333#-81.7...",NaN,"-3.64583333333333,3.38541666666667,7.03125,10....","wc:350,c12.1:42,c12.10:33,c12.12:8,c12.13:13,c...",NaN,<PAGE_PRECISEPUBTIMESTAMP>20171231231600</PAGE...
1,20180101000000-1,20180101000000,1,thestar.com.my,https://www.thestar.com.my/business/business-n...,NaN,1#Malaysia#MY#MY#2.5#112.5#MY;1#Japan#JA#JA#36...,public investment bank;zhejiang geely holding ...,"1.51098901098901,2.47252747252747,0.9615384615...","wc:671,c1.2:9,c1.3:1,c12.1:18,c12.10:59,c12.11...",NaN,<PAGE_LINKS>https://www.thestar.com.my/Search<...
2,20180101000000-2,20180101000000,1,radaronline.com,https://radaronline.com/celebrity-news/bow-wow...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;TAX_WORLD...,1#United States#US#US#39.828175#-98.5795#US,NaN,"4.66101694915254,5.50847457627119,0.8474576271...","wc:206,c12.1:31,c12.10:21,c12.12:6,c12.13:8,c1...",NaN,<PAGE_LINKS>http://radaronline.com/tag/bow-wow...
3,20180101000000-3,20180101000000,1,sanantoniopost.com,http://www.sanantoniopost.com/news/255927392/g...,NaN,"3#Houston, Texas, United States#US#USTX#29.763...",NaN,"-0.704225352112676,1.93661971830986,2.64084507...","wc:527,c12.1:24,c12.10:43,c12.12:19,c12.13:7,c...",NaN,<PAGE_LINKS>http://www.sanantoniopost.com/revi...
4,20180101000000-4,20180101000000,1,complex.com,http://www.complex.com/life/2017/12/dinosaur-e...,NaN,"1#China#CH#CH#35#105#CH;4#Ganzhou, Jiangxi, Ch...",NaN,"-0.495049504950495,1.48514851485149,1.98019801...","wc:187,c1.3:1,c12.1:9,c12.10:10,c12.12:4,c12.1...",NaN,<PAGE_LINKS>http://www.cnn.com/2015/04/21/asia...
...,...,...,...,...,...,...,...,...,...,...,...,...
136619,20180102004500-1796,20180102004500,1,foxtel.com.au,https://community.foxtel.com.au/t5/Foxtel-TV/S...,NaN,NaN,rhithe foxtel community;foxtel,"3.03030303030303,4.04040404040404,1.0101010101...","wc:181,c12.1:13,c12.10:19,c12.12:4,c12.13:4,c1...",NaN,<PAGE_LINKS>http://bit.ly/ContactFoxtel;http:/...
136620,20180102004500-1797,20180102004500,1,forbes.com,https://www.forbes.com/sites/micahsolomon/2018...,TAX_FNCACT;TAX_FNCACT_LEADER;UNGP_FORESTS_RIVE...,NaN,customer service minute,"1.45395799676898,2.74636510500808,1.2924071082...","wc:551,c1.3:3,c12.1:41,c12.10:63,c12.12:14,c12...",NaN,<PAGE_LINKS>http://www.micahsolomon.com;https:...
136621,20180102004500-1798,20180102004500,1,gazette-news.co.uk,http://www.gazette-news.co.uk/news/national/15...,PROTEST;SOC_POINTSOFINTEREST;SOC_POINTSOFINTER...,"4#Tuyserkan, Hamadan, Iran#IR#IR09#34.5481#48....",instagram;obama administration,"-6.12244897959184,2.41187384044527,8.534322820...","wc:497,c1.2:3,c1.3:1,c12.1:22,c12.10:45,c12.12...",NaN,NaN
136622,20180102004500-1799,20180102004500,1,chinadaily.com.cn,http://europe.chinadaily.com.cn/a/201801/02/WS...,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"4#Guangzhou, Guangdong, China#CH#CH30#23.1167#...",home companies airbnb;airbnb inc;china daily i...,"3.2520325203252,3.45528455284553,0.20325203252...","wc:442,c12.1:21,c12.10:34,c12.12:4,c12.13:13,c...",NaN,<PAGE_LINKS>http://europe.chinadaily.com.cn;ht...


In [78]:
df2=df.copy()

# Limit to include United States (removes about half of data)
df2 = df2[df2['V1LOCATIONS'].str.contains('united states', case=False, na=False)]


# Limit to include airlines
df2 = df2[df2['V1ORGANIZATIONS'].str.contains(
    "Airline|Airport|Alaska Airlines|American Airlines|Delta Air Lines|Frontier Airlines|Hawaiian Airlines|JetBlue|Southwest Airlines|Spirit Airlines|Sun Country Airlines|United Airlines|Allegiant Air"
    , case=False, na=False, regex=True)]

df2

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
573,20180101000000-573,20180101000000,1,sys-con.com,http://wearables.sys-con.com/node/4211457,TAX_FNCACT;TAX_FNCACT_DRIVERS;,"3#Lighthouse Point, Florida, United States#US#...",google;big data solutions;devops summit power ...,"1.09239872553482,3.03823395539372,1.9458352298...","wc:7547,c1.2:18,c1.3:10,c1.4:4,c12.1:491,c12.1...",NaN,<PAGE_LINKS>;http://carmengonzalez.sys-con.com...
577,20180101000000-577,20180101000000,1,vcstar.com,http://www.vcstar.com/story/money/nation-now/2...,TAX_FNCACT;TAX_FNCACT_INSIDER;EDUCATION;,"2#Arizona, United States#US#USAZ#33.7712#-111....",frank lloyd wright foundation;arizona biltmore...,"0.966183574879227,1.69082125603865,0.724637681...","wc:801,c1.3:2,c1.4:2,c12.1:42,c12.10:33,c12.12...",NaN,<PAGE_LINKS>http://jobs.vcstar.com;http://www....
765,20180101000000-765,20180101000000,1,abc10.com,http://www.abc10.com/news/nation-world/legal-m...,CRIME_ILLEGAL_DRUGS;TAX_FNCACT;TAX_FNCACT_EVAN...,"2#Maine, United States#US#USME#44.6074#-69.397...",facebook;big tobacco;los angeles international...,"0.0676132521974306,2.77214334009466,2.70453008...","wc:1353,c1.2:11,c1.3:1,c12.1:80,c12.10:130,c12...",NaN,<PAGE_LINKS>https://www.usatoday.com/story/new...
769,20180101000000-769,20180101000000,1,kiwiblog.co.nz,https://www.kiwiblog.co.nz/2018/01/general_deb...,TAX_WORLDARACHNIDS;TAX_WORLDARACHNIDS_TICKS;CR...,1#Mexico#MX#MX#23#-102#MX;1#United States#US#U...,fairport convention;washington post;veterans a...,"-1.22897800776197,3.6222509702458,4.8512289780...","wc:1396,c1.2:1,c1.3:8,c12.1:160,c12.10:150,c12...",NaN,<PAGE_LINKS>;http://cci-reanalyzer.org/wx/Dail...
944,20180101000000-944,20180101000000,1,kiwiblog.co.nz,https://www.kiwiblog.co.nz/2018/01/general_deb...,TAX_WORLDARACHNIDS;TAX_WORLDARACHNIDS_TICKS;CR...,1#Mexico#MX#MX#23#-102#MX;1#United States#US#U...,fairport convention;washington post;veterans a...,"-1.22897800776197,3.6222509702458,4.8512289780...","wc:1396,c1.2:1,c1.3:8,c12.1:160,c12.10:150,c12...",NaN,<PAGE_LINKS>;http://cci-reanalyzer.org/wx/Dail...
...,...,...,...,...,...,...,...,...,...,...,...,...
135832,20180102004500-1009,20180102004500,1,newson6.com,http://www.newson6.com/story/37170869/high-pro...,TRIAL;SOC_GENERALCRIME;KILL;GENERAL_HEALTH;MED...,"2#Oklahoma, United States#US#USOK#35.5376#-96....",tulsa international airport,"-7.38461538461538,1.23076923076923,8.615384615...","wc:291,c12.1:20,c12.10:36,c12.12:17,c12.13:11,...",NaN,<PAGE_AUTHORS>Taylor Newcomb;By: Taylor Newcom...
135979,20180102004500-1156,20180102004500,1,byronnews.com.au,https://www.byronnews.com.au/news/premium-econ...,EPU_ECONOMY_HISTORIC;,"2#New York, United States#US#USNY#42.1497#-74....",new york times;singapore airline;singapore air...,"2.66666666666667,3.93939393939394,1.2727272727...","wc:1480,c1.1:1,c1.2:8,c1.4:1,c12.1:79,c12.10:1...",NaN,<PAGE_LINKS>http://www.singaporeairlines.com/;...
136101,20180102004500-1278,20180102004500,1,startribune.com,http://www.startribune.com/airline-mechanical-...,GENERAL_HEALTH;MEDICAL;SOC_POINTSOFINTEREST;SO...,"3#Boston, Massachusetts, United States#US#USMA...",american airlines;massachusetts port authority...,"-1.48148148148148,0,1.48148148148148,1.4814814...","wc:127,c12.1:1,c12.10:3,c12.12:1,c12.13:1,c12....",NaN,<PAGE_LINKS>http://m.startribune.com/nation/</...
136252,20180102004500-1429,20180102004500,1,cqnews.com.au,https://www.cqnews.com.au/news/premium-economy...,EPU_ECONOMY_HISTORIC;,1#Italy#IT#IT#42.833333#12.833333#IT;2#New Yor...,new york times;singapore airline;singapore air...,"2.65700483091787,3.98550724637681,1.3285024154...","wc:1482,c1.1:1,c1.2:8,c1.4:1,c12.1:82,c12.10:1...",NaN,<PAGE_LINKS>http://www.singaporeairlines.com/;...


In [76]:
# Parse the semicolon-delimited V1ORGANIZATIONS column
df2['V1ORGANIZATIONS'] = df2['V1ORGANIZATIONS'].str.split(';')
# Explode the DataFrame to have one row per organization
df2 = df2.explode('V1ORGANIZATIONS')
# Remove leading and trailing whitespace from the organization names
df2['V1ORGANIZATIONS'] = df2['V1ORGANIZATIONS'].str.strip()
# Remove rows where V1ORGANIZATIONS is NaN or empty
df2 = df2[df2['V1ORGANIZATIONS'].notna() & (df2['V1ORGANIZATIONS'] != '')]



df2

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
1231,20180101001500-7,20180101001500,1,qctimes.com,http://qctimes.com/news/local/davenport-man-re...,TAX_FNCACT;TAX_FNCACT_FATHER;GEN_HOLIDAY;TAX_R...,1#Puerto Rico#RQ#RQ#18.2359#-66.4838#RQ;2#Puer...,emergency management agency,"-0.201816347124117,2.62361251261352,2.82542885...","wc:855,c1.4:1,c12.1:55,c12.10:97,c12.12:39,c12...",NaN,<PAGE_AUTHORS>Jack Cullen;CONTRIBUTED PHOTO;Ja...
1231,20180101001500-7,20180101001500,1,qctimes.com,http://qctimes.com/news/local/davenport-man-re...,TAX_FNCACT;TAX_FNCACT_FATHER;GEN_HOLIDAY;TAX_R...,1#Puerto Rico#RQ#RQ#18.2359#-66.4838#RQ;2#Puer...,delta air lines,"-0.201816347124117,2.62361251261352,2.82542885...","wc:855,c1.4:1,c12.1:55,c12.10:97,c12.12:39,c12...",NaN,<PAGE_AUTHORS>Jack Cullen;CONTRIBUTED PHOTO;Ja...
1231,20180101001500-7,20180101001500,1,qctimes.com,http://qctimes.com/news/local/davenport-man-re...,TAX_FNCACT;TAX_FNCACT_FATHER;GEN_HOLIDAY;TAX_R...,1#Puerto Rico#RQ#RQ#18.2359#-66.4838#RQ;2#Puer...,u s army corps of engineers,"-0.201816347124117,2.62361251261352,2.82542885...","wc:855,c1.4:1,c12.1:55,c12.10:97,c12.12:39,c12...",NaN,<PAGE_AUTHORS>Jack Cullen;CONTRIBUTED PHOTO;Ja...
2231,20180101001500-1007,20180101001500,1,fox26houston.com,http://www.fox26houston.com/news/grandmother-i...,EPU_CATS_REGULATION;TAX_WORLDMAMMALS;TAX_WORLD...,"3#Portland, Oregon, United States#US#USOR#45.5...",portland international airport,"-7.2289156626506,0.602409638554217,7.831325301...","wc:154,c12.1:10,c12.10:14,c12.12:11,c12.13:1,c...",NaN,<PAGE_LINKS>http://www.fox26houston.com/news/g...
2231,20180101001500-1007,20180101001500,1,fox26houston.com,http://www.fox26houston.com/news/grandmother-i...,EPU_CATS_REGULATION;TAX_WORLDMAMMALS;TAX_WORLD...,"3#Portland, Oregon, United States#US#USOR#45.5...",alaska airlines,"-7.2289156626506,0.602409638554217,7.831325301...","wc:154,c12.1:10,c12.10:14,c12.12:11,c12.13:1,c...",NaN,<PAGE_LINKS>http://www.fox26houston.com/news/g...
...,...,...,...,...,...,...,...,...,...,...,...,...
135761,20180102004500-938,20180102004500,1,kctv5.com,http://www.kctv5.com/story/37170886/mechanical...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;SOC_POINT...,"3#Boston, Massachusetts, United States#US#USMA...",massachusetts port authority,"-1.45985401459854,0,1.45985401459854,1.4598540...","wc:133,c12.1:3,c12.10:5,c12.12:1,c12.13:2,c12....",NaN,<PAGE_LINKS>http://www.wcvb.com/article/4-peop...
135761,20180102004500-938,20180102004500,1,kctv5.com,http://www.kctv5.com/story/37170886/mechanical...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;SOC_POINT...,"3#Boston, Massachusetts, United States#US#USMA...",associated press,"-1.45985401459854,0,1.45985401459854,1.4598540...","wc:133,c12.1:3,c12.10:5,c12.12:1,c12.13:2,c12....",NaN,<PAGE_LINKS>http://www.wcvb.com/article/4-peop...
136101,20180102004500-1278,20180102004500,1,startribune.com,http://www.startribune.com/airline-mechanical-...,GENERAL_HEALTH;MEDICAL;SOC_POINTSOFINTEREST;SO...,"3#Boston, Massachusetts, United States#US#USMA...",american airlines,"-1.48148148148148,0,1.48148148148148,1.4814814...","wc:127,c12.1:1,c12.10:3,c12.12:1,c12.13:1,c12....",NaN,<PAGE_LINKS>http://m.startribune.com/nation/</...
136101,20180102004500-1278,20180102004500,1,startribune.com,http://www.startribune.com/airline-mechanical-...,GENERAL_HEALTH;MEDICAL;SOC_POINTSOFINTEREST;SO...,"3#Boston, Massachusetts, United States#US#USMA...",massachusetts port authority,"-1.48148148148148,0,1.48148148148148,1.4814814...","wc:127,c12.1:1,c12.10:3,c12.12:1,c12.13:1,c12....",NaN,<PAGE_LINKS>http://m.startribune.com/nation/</...


In [77]:
df2['V1ORGANIZATIONS'].value_counts().head(20)

V1ORGANIZATIONS
american airlines                                  79
united states                                      56
hawaiian airlines                                  46
twitter                                            41
massachusetts port authority                       40
delta air lines                                    38
united airlines                                    33
associated press                                   31
washington post                                    23
southwest airlines                                 23
facebook                                           19
nasdaq                                             18
alaska airlines                                    18
virgin america                                     16
jetblue                                            15
morgan stanley                                     14
university of kentucky                             13
white house                                        13
fort worth i

In [73]:
df4=df2[df2['V1ORGANIZATIONS']=='american airlines'].copy()
df4

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
3612,20180101003000-1053,20180101003000,1,wqcmfm.com,http://www.wqcmfm.com/syndicated-article/?id=5...,NaN,"3#Sioux Falls, South Dakota, United States#US#...",american airlines,"-4.7808764940239,0.398406374501992,5.179282868...","wc:258,c12.1:14,c12.10:13,c12.12:9,c12.13:2,c1...",NaN,NaN
4200,20180101004500-127,20180101004500,1,khar590.com,http://www.khar590.com/syndicated-article/?id=...,NaN,"3#Sioux Falls, South Dakota, United States#US#...",american airlines,"-4.7808764940239,0.398406374501992,5.179282868...","wc:258,c12.1:14,c12.10:13,c12.12:9,c12.13:2,c1...",NaN,NaN
4552,20180101004500-479,20180101004500,1,1640thechamp.com,http://www.1640thechamp.com/syndicated-article...,NaN,"3#Sioux Falls, South Dakota, United States#US#...",american airlines,"-4.7808764940239,0.398406374501992,5.179282868...","wc:258,c12.1:14,c12.10:13,c12.12:9,c12.13:2,c1...",NaN,NaN
6442,20180101010000-806,20180101010000,1,thezone1059.com,http://www.thezone1059.com/syndicated-article/...,NaN,"3#Sioux Falls, South Dakota, United States#US#...",american airlines,"-3.96825396825397,0.396825396825397,4.36507936...","wc:255,c12.1:14,c12.10:15,c12.12:8,c12.13:4,c1...",NaN,NaN
9563,20180101013000-904,20180101013000,1,rockthefox.com,http://www.rockthefox.com/syndicated-article/?...,NaN,"3#Sioux Falls, South Dakota, United States#US#...",american airlines,"-4.7808764940239,0.398406374501992,5.179282868...","wc:258,c12.1:14,c12.10:13,c12.12:9,c12.13:2,c1...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
134465,20180102003000-1263,20180102003000,1,am930theanswer.com,http://am930theanswer.com/news/national/airlin...,GENERAL_HEALTH;MEDICAL;SOC_POINTSOFINTEREST;SO...,"3#Boston, Massachusetts, United States#US#USMA...",american airlines,"-1.40845070422535,0,1.40845070422535,1.4084507...","wc:130,c12.1:1,c12.10:3,c12.12:1,c12.13:1,c12....",NaN,<PAGE_LINKS>http://am930theanswer.com/news/nat...
134768,20180102003000-1566,20180102003000,1,newsadvance.com,http://www.newsadvance.com/news/national/wire/...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;SOC_POINT...,"3#Boston, Massachusetts, United States#US#USMA...",american airlines,"-1.48148148148148,0,1.48148148148148,1.4814814...","wc:130,c12.1:3,c12.10:5,c12.12:1,c12.13:2,c12....",NaN,<PAGE_LINKS>http://www.wcvb.com/article/4-peop...
135711,20180102004500-888,20180102004500,1,americanlivewire.com,http://americanlivewire.com/2013-12-14-1-foot-...,NaN,"3#Carrabassett, Maine, United States#US#USME#4...",american airlines,"-1.61507402422611,1.74966352624495,3.364737550...","wc:684,c1.2:1,c12.1:35,c12.10:66,c12.12:21,c12...",NaN,<PAGE_LINKS>http://americanlivewire.com/author...
135761,20180102004500-938,20180102004500,1,kctv5.com,http://www.kctv5.com/story/37170886/mechanical...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;SOC_POINT...,"3#Boston, Massachusetts, United States#US#USMA...",american airlines,"-1.45985401459854,0,1.45985401459854,1.4598540...","wc:133,c12.1:3,c12.10:5,c12.12:1,c12.13:2,c12....",NaN,<PAGE_LINKS>http://www.wcvb.com/article/4-peop...


In [70]:
df3=pd.DataFrame(df2['V1ORGANIZATIONS'].value_counts().reset_index())
df3

,V1ORGANIZATIONS,count
0,united states,11857
1,associated press,8914
2,twitter,6368
3,facebook,4249
4,cnn,2984
...,...,...
44959,seaman high school,1
44960,university of alabama birmingham,1
44961,florence events center,1
44962,georgia southern university jiann,1
